# CourseData ETL

## STEP 1: Load SQL and create CourseData database

In [27]:
%load_ext sql
import pandas as pd
import sqlite3

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [28]:
%sql sqlite:///DBTest.db
conn = sqlite3.connect('DBTest.db')

## STEP 2: Create tables as designed in Lucidchart

In [29]:
%%sql

DROP TABLE IF EXISTS PROFESSORS;
CREATE TABLE PROFESSORS (
    Professor_id INTEGER PRIMARY KEY,
    Name TEXT NOT NULL
);

 * sqlite:///DBTest.db
(sqlite3.OperationalError) attempt to write a readonly database
[SQL: DROP TABLE IF EXISTS PROFESSORS;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [30]:
%%sql
DROP TABLE IF EXISTS PROGRAMS;
CREATE TABLE PROGRAMS (
    Program_id INTEGER PRIMARY KEY,
    program_code TEXT(2) NOT NULL,
    program_name TEXT NOT NULL
);

 * sqlite:///DBTest.db
(sqlite3.OperationalError) attempt to write a readonly database
[SQL: DROP TABLE IF EXISTS PROGRAMS;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [31]:
%%sql
DROP TABLE IF EXISTS LOCATIONS;
CREATE TABLE LOCATIONS (
    Location_id INTEGER PRIMARY KEY,
    location TEXT NOT NULL
);

 * sqlite:///DBTest.db
(sqlite3.OperationalError) attempt to write a readonly database
[SQL: DROP TABLE IF EXISTS LOCATIONS;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [32]:
%%sql
DROP TABLE IF EXISTS COURSES;
CREATE TABLE COURSES (
    Course_id INTEGER PRIMARY KEY, 
    CatalogYear TEXT NOT NULL,
    Catalog_id TEXT NOT NULL,
    Course_Title TEXT NOT NULL,
    Credits TEXT NOT NULL,
    Attributes TEXT,
    Prereqs TEXT,
    Coreqs TEXT,
    Description TEXT,
    Fee TEXT,
    Program_id INTEGER NOT NULL,
    FOREIGN KEY (Program_id) REFERENCES PROGRAMS(Program_id)
);

 * sqlite:///DBTest.db
(sqlite3.OperationalError) attempt to write a readonly database
[SQL: DROP TABLE IF EXISTS COURSES;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [33]:
%%sql
DROP TABLE IF EXISTS COURSE_OFFERINGS;
CREATE TABLE COURSE_OFFERINGS (
    Offering_id INTEGER PRIMARY KEY,
    Term TEXT,
    Section TEXT,
    Crn INTEGER NOT NULL,
    Cap INTEGER NOT NULL,
    Actual INTEGER NOT NULL,
    Title TEXT NOT NULL
    Remaining INTEGER NOT NULL,
    Timecodes TEXT,
    Course_id INTEGER NOT NULL,
    Catalog_id TEXT NOT NULL,
    Program_id INTEGER NOT NULL,
    Name TEXT NOT NULL,
    FOREIGN KEY (Name) REFERENCES PROFESSORS(Name)
    FOREIGN KEY (Program_id) REFERENCES PROGRAMS(Program_id)
    FOREIGN KEY (Course_id) REFERENCES COURSES(Course_id)
);

 * sqlite:///DBTest.db
Done.
(sqlite3.OperationalError) near "Remaining": syntax error
[SQL: CREATE TABLE COURSE_OFFERINGS (
    Offering_id INTEGER PRIMARY KEY,
    Term TEXT,
    Section TEXT,
    Crn INTEGER NOT NULL,
    Cap INTEGER NOT NULL,
    Actual INTEGER NOT NULL,
    Title TEXT NOT NULL
    Remaining INTEGER NOT NULL,
    Timecodes TEXT,
    Course_id INTEGER NOT NULL,
    Catalog_id TEXT NOT NULL,
    Program_id INTEGER NOT NULL,
    Name TEXT NOT NULL,
    FOREIGN KEY (Name) REFERENCES PROFESSORS(Name)
    FOREIGN KEY (Program_id) REFERENCES PROGRAMS(Program_id)
    FOREIGN KEY (Course_id) REFERENCES COURSES(Course_id)
);]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [8]:
%%sql
DROP TABLE IF EXISTS MEETINGS;
CREATE TABLE MEETINGS (
    Meetings_id INTEGER PRIMARY KEY,
    Day TEXT,
    Start TEXT NOT NULL,
    End TEXT NOT NULL,
    Location_id INTEGER NOT NULL,
    Offering_id INTEGER,
    FOREIGN KEY (Location_id) REFERENCES LOCATIONS(Location_id),
    FOREIGN KEY (Offering_id) REFERENCES COURSE_OFFERINGS(Offering_id)
);

 * sqlite:///DBTest.db
Done.
Done.


[]

In [9]:
%%sql
DROP TABLE IF EXISTS YEAR;
CREATE TABLE YEAR (
    Year TEXT NOT NULL,
    Term TEXT
);

 * sqlite:///DBTest.db
Done.
Done.


[]

In [10]:
%%sql
INSERT INTO YEAR (Term, Year) VALUES 
('Fall2017','2017_2018'),
('Winter2018','2017_2018'),
('Spring2018','2017_2018'),
('Summer2018','2017_2018'),
('Fall2018','2018_2019'),
('Winter2019','2018_2019'),
('Spring2019','2018_2019');

 * sqlite:///DBTest.db
7 rows affected.


[]

In [11]:
%%sql
SELECT * FROM YEAR
LIMIT (2);

 * sqlite:///DBTest.db
Done.


Year,Term
2017_2018,Fall2017
2017_2018,Winter2018


## STEP 3: Import CSV files

In [12]:
# importing course catalog csvs (taken from professsor's example )
course_catalog = ['2017_2018', '2018_2019']

# loop using pandas
for cat_year in course_catalog:
    path = 'SourceData/Catalogs/CourseCatalog'+cat_year+'.csv' # path to directory
    data = pd.read_csv(path) # read all csvs in path
    data['cat_year'] = cat_year # add new column 'cat_year' to differentiate between 2017-18, 2018-19
    data.to_sql('IMPORT_CATALOG_COURSES',conn,if_exists='append',index=False) # converting to sql & unique

In [13]:
# meetings df

semesters = ['Fall2014','Fall2015','Fall2016','Fall2017','Fall2018','Spring2015', 'Spring2016','Spring2017','Spring2018','Spring2019','SpringBreak2017',
                   'Summer2015', 'Summer2016','Summer2017','Summer2018','Winter2015','Winter2016', 'Winter2017','Winter2018']

for semester in semesters: 
    path = 'SourceData/'+semester+'/course_meetings.csv'
    data = pd.read_csv(path)
    data.to_sql('IMPORT_COURSE_MEETINGS',conn,if_exists='append',index=False) 

In [25]:
# courses df
semesters = ['Fall2014','Fall2015','Fall2016','Fall2017','Fall2018','Spring2015', 'Spring2016','Spring2017','Spring2018','Spring2019','SpringBreak2017',
                   'Summer2015', 'Summer2016','Summer2017','Summer2018','Winter2015','Winter2016', 'Winter2017','Winter2018']

for semester in semesters:
    path = 'SourceData/'+semester+'/courses.csv'
    data = pd.read_csv(path)
    data['cat_year'] = cat_year # add new column 'cat_year' to differentiate between 2017-18, 2018-19
    data.to_sql('IMPORT_COURSE_OFFERINGS',conn,if_exists='append',index=False) 

OperationalError: table IMPORT_COURSE_OFFERINGS has no column named cat_year

In [15]:
%%sql
--check for duplication
SELECT 
    (SELECT Count(*) FROM IMPORT_COURSE_OFFERINGS) as 'Count',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_COURSE_OFFERINGS)) as 'DistinctCount';

 * sqlite:///DBTest.db
Done.


Count,DistinctCount
15937,15937


In [16]:
%%sql
-- check for duplication in course meetings

SELECT 
    (SELECT Count(*) FROM IMPORT_COURSE_MEETINGS) as 'Count',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_COURSE_MEETINGS)) as 'DistinctCount';

 * sqlite:///DBTest.db
Done.


Count,DistinctCount
284907,284847


In [17]:
%%sql
--check for duplication in catalog courses

SELECT 
    (SELECT Count(*) FROM IMPORT_CATALOG_COURSES) as 'Count',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_CATALOG_COURSES)) as 'DistinctCount';

 * sqlite:///DBTest.db
Done.


Count,DistinctCount
4440,4440


## STEP 4: POPULATING THE TABLES

In [18]:
%%sql
DELETE FROM PROFESSORS;
INSERT INTO PROFESSORS(Name)
    SELECT DISTINCT primary_instructor
    FROM IMPORT_COURSE_OFFERINGS;
SELECT * FROM PROFESSORS LIMIT 5;

 * sqlite:///DBTest.db
0 rows affected.
1104 rows affected.
Done.


Professor_id,Name
1,Michael P. Coyne
2,Rebecca I. Bloch
3,Paul Caster
4,Jo Ann Drusbosky
5,Arleen N. Kardos


In [19]:
%%sql
DELETE FROM PROGRAMS;
INSERT INTO PROGRAMS(program_code, program_name)
    SELECT DISTINCT program_code, program_name
    FROM IMPORT_CATALOG_COURSES;
SELECT * FROM PROGRAMS LIMIT 5;

 * sqlite:///DBTest.db
0 rows affected.
83 rows affected.
Done.


Program_id,program_code,program_name
1,AN,Asian Studies
2,BU,Business
3,BL,Black Studies
4,BEN,Bioengineering
5,AR,Arabic


In [21]:
%%sql
DELETE FROM COURSES;
INSERT INTO COURSES(CatalogYear, Catalog_id, Course_Title, Credits, Attributes, Prereqs, Coreqs, Description, Fee, Program_id)
    SELECT DISTINCT cat_year, Catalog_id, course_title, credits, attributes, prereqs, coreqs, description, fees, Program_id 
    FROM IMPORT_CATALOG_COURSES
        JOIN PROGRAMS USING (Program_code);
SELECT * FROM COURSES LIMIT 2;

 * sqlite:///DBTest.db
0 rows affected.
4440 rows affected.
Done.


Course_id,CatalogYear,Catalog_id,Course_Title,Credits,Attributes,Prereqs,Coreqs,Description,Fee,Program_id
1,2017_2018,AN 0301,Independent Study,1-3 Credits,None,None,None,Students undertake an individualized program of study in consultation with a director from the Asian studies faculty.,None,1
2,2017_2018,AN 0310,Asian Studies Seminar,3 Credits,None,None,None,"This seminar examines selected topics concerning Asia. This course is taught in conjunction with another 100-300 level course from a rotation of course offerings. Consult the Asian Studies director to identify the conjoined course for a given semester. The seminar concentrates on topics within the parameters of the conjoined course syllabus but adds research emphasis. Students registered for this course must complete a research project, to include 300-level research, in addition to the regular research requirements of the conjoined course, and a 25-50 page term paper in substitution of some portion of the conjoined course requirements, as determined by the instructor. Open to juniors and seniors only.",None,1


In [23]:
%%sql
DELETE FROM COURSES;
INSERT INTO COURSES(CatalogYear, Catalog_id, Course_Title, Credits, Attributes, Prereqs, Coreqs, Description, Fee, Program_id)
    SELECT DISTINCT cat_year, Catalog_id, course_title, credits, attributes, prereqs, coreqs, description, fees, Program_id 
    FROM IMPORT_CATALOG_COURSES
        JOIN PROGRAMS USING (program_code);
SELECT * FROM COURSES LIMIT 2;

 * sqlite:///DBTest.db
0 rows affected.
4440 rows affected.
Done.


Course_id,CatalogYear,Catalog_id,Course_Title,Credits,Attributes,Prereqs,Coreqs,Description,Fee,Program_id
1,2017_2018,AN 0301,Independent Study,1-3 Credits,None,None,None,Students undertake an individualized program of study in consultation with a director from the Asian studies faculty.,None,1
2,2017_2018,AN 0310,Asian Studies Seminar,3 Credits,None,None,None,"This seminar examines selected topics concerning Asia. This course is taught in conjunction with another 100-300 level course from a rotation of course offerings. Consult the Asian Studies director to identify the conjoined course for a given semester. The seminar concentrates on topics within the parameters of the conjoined course syllabus but adds research emphasis. Students registered for this course must complete a research project, to include 300-level research, in addition to the regular research requirements of the conjoined course, and a 25-50 page term paper in substitution of some portion of the conjoined course requirements, as determined by the instructor. Open to juniors and seniors only.",None,1


In [24]:
%%sql
DELETE FROM LOCATIONS;
INSERT INTO LOCATIONS(location)
    SELECT DISTINCT location
    FROM IMPORT_COURSE_MEETINGS
    ORDER BY location;
SELECT * FROM LOCATIONS LIMIT 5;

 * sqlite:///DBTest.db
0 rows affected.
207 rows affected.
Done.


Location_id,location
1,BCC 200
2,BD
3,BH
4,BH BY ARR
5,BLM 112


In [ ]:
%%sql
DELETE FROM COURSE_OFFERINGS;
INSERT INTO COURSE_OFFERINGS(Term, Cap, Section, Crn, Actual, Remaining, Timecodes, Course_id, Name)
    SELECT DISTINCT term, cap, section, crn, act, rem, timecodes, Course_id, primary_instructor 
    FROM IMPORT_COURSE_OFFERINGS
        JOIN PROFESSORS ON (IMPORT_COURSE_OFFERINGS.primary_instructor = PROFESSORS.Name)
        JOIN COURSES ON (IMPORT_COURSE_OFFERINGS.Catalog_id= COURSES.Catalog_id)
        JOIN PROGRAMS USING (Program_id);
SELECT * FROM COURSE_OFFERINGS LIMIT 5;

In [ ]:
%%sql
DELETE FROM MEETINGS;
INSERT INTO MEETINGS(Meetings, Day, Start, End, Location_id)
    SELECT DISTINCT Meetings, day, start, end, Location_id
    FROM IMPORT_CATALOG_COURSES
        JOIN COURSE_OFFERINGS USING (Term, Crn) 
        LEFT JOIN LOCATIONS ON (LOCATIONS.location_id = IMPORT_CATALOG_COURSES.location);
SELECT * FROM MEETINGS LIMIT 5;  